In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#import congitive assessment spreadsheet
#path = "./Data/matthewl225_7_15_2019_16_11_22.csv"
path = input('Path to cognitive assesment csv: ')
header = ['ADRC_ADRCCLINICALDATA ID', 'dx1']

#dfcolumns = pd.read_csv(path, nrows = 1)
df = pd.read_csv(path, header = None, skiprows = 1, usecols = [0,8], names = header)

#df = pd.read_csv(path, names=header, index_col=False)

Path to cognitive assesment csv: ./Data/matthewl225_7_15_2019_16_11_22.csv


In [4]:
df.shape

(6224, 2)

In [5]:
df[:3]

,ADRC_ADRCCLINICALDATA ID,dx1
0,OAS30001_ClinicalData_d0000,Cognitively normal
1,OAS30001_ClinicalData_d0339,Cognitively normal
2,OAS30001_ClinicalData_d0722,Cognitively normal


In [6]:
df.dx1.unique()

array(['Cognitively normal', 'uncertain dementia',
       'Unc: ques. Impairment', 'AD dem w/depresss- not contribut',
       'AD Dementia', 'AD dem distrubed social- with',
       'AD dem w/CVD contribut', 'Non AD dem- Other primary',
       'AD dem visuospatial- with', 'AD dem visuospatial- prior',
       'AD dem Language dysf after', '.',
       'uncertain- possible NON AD dem', 'Vascular Demt- primary',
       'AD dem w/PDI after AD dem not contrib', '0.5 in memory only',
       'Incipient Non-AD dem', 'AD dem distrubed social- prior',
       'Frontotemporal demt. prim', 'AD dem distrubed social- after',
       'AD dem w/PDI after AD dem contribut',
       'uncertain  possible NON AD dem',
       'AD dem w/depresss  not contribut',
       'AD dem w/oth (list B) contribut', 'DLBD- primary',
       'Incipient demt PTP', 'Dementia/PD- primary',
       'AD dem w/depresss- contribut', 'Unc: impair reversible',
       'AD dem w/oth (list B) not contrib', 'AD dem w/CVD not contrib',
     

In [7]:
df.dx1.isna().sum()

2

In [8]:
import re

subject_regex = re.compile("OAS(?P<order>[0-9]+)")
subjects = [subject_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
#print(subjects)
df['Subject'] = pd.Series(subjects, index=df.index)

date_regex = re.compile("d(?P<order>[0-9]+)")
dates = [date_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
df['Date'] = pd.Series(list(map(int,dates)), index=df.index)

In [9]:
df[:15]

,ADRC_ADRCCLINICALDATA ID,dx1,Subject,Date
0,OAS30001_ClinicalData_d0000,Cognitively normal,30001,0
1,OAS30001_ClinicalData_d0339,Cognitively normal,30001,339
2,OAS30001_ClinicalData_d0722,Cognitively normal,30001,722
3,OAS30001_ClinicalData_d1106,Cognitively normal,30001,1106
4,OAS30001_ClinicalData_d1456,Cognitively normal,30001,1456
5,OAS30001_ClinicalData_d1894,Cognitively normal,30001,1894
6,OAS30001_ClinicalData_d2181,Cognitively normal,30001,2181
7,OAS30001_ClinicalData_d2699,Cognitively normal,30001,2699
8,OAS30001_ClinicalData_d3025,Cognitively normal,30001,3025
9,OAS30001_ClinicalData_d3332,Cognitively normal,30001,3332


In [10]:
import scipy.misc, numpy, shutil, os, nibabel
import sys, getopt
import zipfile, os

In [11]:
#unzip nii package
package_path = input('Path to nii zip: ')
with zipfile.ZipFile(package_path, "r") as zip_ref:
    zip_ref.extractall("./Data/Nii")

Path to nii zip: ./Data/matthewl225-20190723_172030.zip


In [12]:
import gzip

#unzip individual run nii pakcages
nii_package_path = "./Data/Nii"
output_path = "./Data/Nii2"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    print("Created ouput directory: " + output_path)

for root, directories, filenames in os.walk(nii_package_path):
    for filename in filenames:
        run_package = os.path.join(root, filename)
        with gzip.open(run_package, 'rb') as s_file, open(os.path.join(output_path, filename[:-3]), 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file, 65536)
            #shutil.move(d_file, output_path)
                

Created ouput directory: ./Data/Nii2


In [13]:
shutil.rmtree("./Data/Nii")

In [15]:
#convert nii to png (borrowed from nii2png.py)
outputfile = "./Data/Images"
rotate_angle = int(input('Enter rotate angle (0 90 180 270): '))

if not (rotate_angle == 0 or rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270):
    print('You must enter a value that is either 90, 180, or 270. Quitting...')
    sys.exit()

data_path = "./Data/Nii2"
for root, directories, filenames in os.walk(data_path):
    for filename in filenames:
        if "T2" in filename or "echo" in filename:
            print(filename)
            continue
        
        nii_path = os.path.join(root, filename)
        image_array = nibabel.load(nii_path).get_data()
        print("Input file is ", nii_path)
        #print(len(image_array.shape))

        if len(image_array.shape) == 3:
            # set 4d array dimension values
            nx, ny, nz = image_array.shape

            if not os.path.exists(outputfile):
                os.makedirs(outputfile)
                print("Created ouput directory: " + outputfile)
            
            image_folder = os.path.join(outputfile, filename[:-8])
            
            if not os.path.exists(image_folder):
                os.makedirs(image_folder)
                print("Created ouput directory: " + image_folder)
            
            print('Reading NIfTI file...')

            total_slices = image_array.shape[2]

            slice_counter = 0
            # iterate through slices
            for current_slice in range(0, total_slices):
                # alternate slices
                if (slice_counter % 1) == 0:
                    # rotate or no rotate
                    if rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270:
                        if rotate_angle == 90:
                            data = numpy.rot90(image_array[:, :, current_slice])
                        elif ask_rotate_num == 180:
                            data = numpy.rot90(numpy.rot90(image_array[:, :, current_slice]))
                        elif ask_rotate_num == 270:
                            data = numpy.rot90(numpy.rot90(numpy.rot90(image_array[:, :, current_slice])))
                    else:
                        data = image_array[:, :, current_slice]

                    #alternate slices and save as png
                    if (slice_counter % 1) == 0:
                        #print('Saving image...')
                        image_name = nii_path[:-4] + "_z" + "{:0>3}".format(str(current_slice+1))+ ".png"
                        scipy.misc.imsave(image_name, data)
                        #print('Saved.')

                        #move images to folder
                        #print('Moving image...')
                        src = image_name
                        shutil.move(src, image_folder)
                        slice_counter += 1
                        #print('Moved.')

            print('Finished converting {}'.format(filename))
        else:
            print('Not a 3D Image. Please try again.')
        




Enter rotate angle (0 90 180 270): 90
Input file is  ./Data/Nii2\sub-OAS30009_ses-d0148_run-01_T1w.nii
Created ouput directory: ./Data/Images
Created ouput directory: ./Data/Images\sub-OAS30009_ses-d0148_run-01
Reading NIfTI file...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Finished converting sub-OAS30009_ses-d0148_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30009_ses-d0148_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30009_ses-d0148_run-02
Reading NIfTI file...
Finished converting sub-OAS30009_ses-d0148_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30009_ses-d1210_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30009_ses-d1210
Reading NIfTI file...
Finished converting sub-OAS30009_ses-d1210_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30009_ses-d2457_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30009_ses-d2457
Reading NIfTI file...
Finished converting sub-OAS30009_ses-d2457_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30019_ses-d0376_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30019_ses-d0376
Reading NIfTI file...
Finished converting sub-OAS30019_ses-d0376_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30029_ses-d0131_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30029_ses-d0131
Reading NIfTI file...
Finishe

Created ouput directory: ./Data/Images\sub-OAS30319_ses-d0043_run-02
Reading NIfTI file...
Finished converting sub-OAS30319_ses-d0043_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30329_ses-d0376_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30329_ses-d0376
Reading NIfTI file...
Finished converting sub-OAS30329_ses-d0376_T1w.nii
sub-OAS30339_ses-d0066_echo-1_run-03_T1w.nii
sub-OAS30339_ses-d0066_echo-2_run-03_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30339_ses-d0066_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30339_ses-d0066_run-01
Reading NIfTI file...
Finished converting sub-OAS30339_ses-d0066_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30339_ses-d0066_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30339_ses-d0066_run-02
Reading NIfTI file...
Finished converting sub-OAS30339_ses-d0066_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30349_ses-d0699_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30349_ses-d0699_run-01
Readin

Created ouput directory: ./Data/Images\sub-OAS30579_ses-d2400
Reading NIfTI file...
Finished converting sub-OAS30579_ses-d2400_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30589_ses-d0096_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30589_ses-d0096_run-01
Reading NIfTI file...
Finished converting sub-OAS30589_ses-d0096_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30589_ses-d0096_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30589_ses-d0096_run-02
Reading NIfTI file...
Finished converting sub-OAS30589_ses-d0096_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30589_ses-d1525_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30589_ses-d1525
Reading NIfTI file...
Finished converting sub-OAS30589_ses-d1525_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30589_ses-d2311_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30589_ses-d2311
Reading NIfTI file...
Finished converting sub-OAS30589_ses-d2311_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30589_ses-d3191_T

Input file is  ./Data/Nii2\sub-OAS30869_ses-d0077_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30869_ses-d0077_run-02
Reading NIfTI file...
Finished converting sub-OAS30869_ses-d0077_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30869_ses-d1691_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30869_ses-d1691
Reading NIfTI file...
Finished converting sub-OAS30869_ses-d1691_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30869_ses-d2290_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30869_ses-d2290
Reading NIfTI file...
Finished converting sub-OAS30869_ses-d2290_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30879_ses-d3497_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30879_ses-d3497
Reading NIfTI file...
Finished converting sub-OAS30879_ses-d3497_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30899_ses-d0070_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30899_ses-d0070_run-01
Reading NIfTI file...
Finished converting sub-OAS30899_ses-d0070_run-01_T

Created ouput directory: ./Data/Images\sub-OAS31109_ses-d0000_run-02
Reading NIfTI file...
Finished converting sub-OAS31109_ses-d0000_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31109_ses-d0000_run-03_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31109_ses-d0000_run-03
Reading NIfTI file...
Finished converting sub-OAS31109_ses-d0000_run-03_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31119_ses-d0069_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31119_ses-d0069
Reading NIfTI file...
Finished converting sub-OAS31119_ses-d0069_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31119_ses-d1493_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31119_ses-d1493
Reading NIfTI file...
Finished converting sub-OAS31119_ses-d1493_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31129_ses-d0055_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31129_ses-d0055
Reading NIfTI file...
Finished converting sub-OAS31129_ses-d0055_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31139_ses-d0533_T1w.nii


In [18]:
runs_list = next(os.walk('./Data/Images'))[1]
#print(runs_list)

['sub-OAS30009_ses-d0148_run-01', 'sub-OAS30009_ses-d0148_run-02', 'sub-OAS30009_ses-d1210', 'sub-OAS30009_ses-d2457', 'sub-OAS30019_ses-d0376', 'sub-OAS30029_ses-d0131', 'sub-OAS30029_ses-d0893', 'sub-OAS30039_ses-d0103', 'sub-OAS30049_ses-d0013_run-01', 'sub-OAS30049_ses-d0013_run-02', 'sub-OAS30049_ses-d1633', 'sub-OAS30059_ses-d0230_run-01', 'sub-OAS30059_ses-d0230_run-02', 'sub-OAS30059_ses-d0230_run-03', 'sub-OAS30059_ses-d0230_run-04', 'sub-OAS30059_ses-d1188', 'sub-OAS30079_ses-d0019_run-01', 'sub-OAS30079_ses-d0019_run-02', 'sub-OAS30079_ses-d1833', 'sub-OAS30079_ses-d2668', 'sub-OAS30089_ses-d0001', 'sub-OAS30099_ses-d0032', 'sub-OAS30109_ses-d0270', 'sub-OAS30109_ses-d0432_run-01', 'sub-OAS30109_ses-d0432_run-02', 'sub-OAS30109_ses-d0997_run-01', 'sub-OAS30109_ses-d0997_run-02', 'sub-OAS30109_ses-d2310', 'sub-OAS30109_ses-d2320', 'sub-OAS30119_ses-d1209_run-01', 'sub-OAS30119_ses-d1209_run-02', 'sub-OAS30119_ses-d1209_run-03', 'sub-OAS30119_ses-d2415_run-01', 'sub-OAS30119_s

In [19]:
import math

AD_list = []

for run in runs_list:
    # if dx1 starts with AD then patient has AD
    # either 0 or 1
    # print(run)
    subject_regex = re.compile("OAS(?P<order>[0-9]+)")
    subject = subject_regex.search(run).group(1)
    
    date_regex = re.compile("d(?P<order>[0-9]+)")
    date = date_regex.search(run).group(1)
    
    #print(subject, date)
    
    subject_df = df.loc[df["Subject"] == subject]
    #print(subject_df)
    
    #label = min(list(map(int, subject_df['Date'])), key=lambda x:(int(date)-x))
    df_dates = subject_df['Date']
    #print(type(df_dates))
    
    if int(date) > df_dates.iloc[-1]:
        label_date = df_dates.iloc[-1]
    else:
        label_date = min(i for i in df_dates if i >= int(date))
    
    label = subject_df.loc[df['Date'] == label_date, 'dx1'].item()
    if type(label) == float:
        print(label)
        runs_list.remove(run)
        continue
    AD = 1 if label.startswith('AD') else 0
    AD_list.append(AD)
    
    #print(run, AD)

Labelsdf = pd.DataFrame()
#df = pd.DataFrame(columns=['Run', 'AD'])
Labelsdf['Run'] = pd.Series(runs_list)
Labelsdf['AD'] = pd.Series(AD_list)

In [20]:
print(label)

AD Dementia


In [21]:
Labelsdf

,Run,AD
0,sub-OAS30009_ses-d0148_run-01,0
1,sub-OAS30009_ses-d0148_run-02,0
2,sub-OAS30009_ses-d1210,0
3,sub-OAS30009_ses-d2457,0
4,sub-OAS30019_ses-d0376,1
5,sub-OAS30029_ses-d0131,1
6,sub-OAS30029_ses-d0893,1
7,sub-OAS30039_ses-d0103,0
8,sub-OAS30049_ses-d0013_run-01,0
9,sub-OAS30049_ses-d0013_run-02,0


In [22]:
Labelsdf.to_csv("./Data/Labels.csv")

In [23]:
Labelsdf.groupby("AD").size()

AD
0    269
1     61
dtype: int64